## Example: CLD on Sleep and Depressive symptoms
In this example, we  implement a system dynamics model (SDM) analysis of a causal loop diagram (CLD) published as an example in the Education Corner article by Uleman et al (2024) doi: 10.1093/ije/dyae091. This CLD focuses on the feedback loops between sleep and depressive symptoms in young adults.

In Kumu, the diagram looks as follows:

<img src="Kumu/Sleep CLD.jpg" alt="drawing" width="800"/>

To analyze this CLD, we will first run the initialization file, which imports the settings and the Kumu file (should have the same name). We will then run the simulations with what-if scenarios, assess the intervention effects, conduct the feedback loop analysis, and perform sensitivity analysis over the model parameters.

In [1]:
import systemdynamics as sd

setting_name = "Sleep"  # Specify the relevant name of the CLD (Excel and json file should match this name)  
extract = sd.Extract(setting_name)  # Load the relevant Kumu file extraction module
s = extract.extract_settings()  # Extract the settings using the json file and the Kumu table
sdm = sd.SDM(s.df_adj, s.interactions_matrix, s)  # Load the module for formulating and simulating the SDM

## Print the main simulating settings
for sett in ["N", "time_unit", "t_end", "dt", "variable_of_interest", "max_parameter_value",
             "double_factor_interventions", "solve_analytically"]:
    print(sett, "=", vars(s)[sett])


AttributeError: module 'systemdynamics' has no attribute 'Extract'

Given these settings, we can now run simulations from the SDM-converted CLD.

In [ ]:
df_sol, param_samples = sdm.run_simulations()  # Run the simulated interventions
intervention_effects = sdm.get_intervention_effects()  # Get the intervention effects
top_plot = None  # Number of top interventions to plot
sd.plots.plot_simulated_intervention_ranking(s, intervention_effects, top_plot=top_plot);  # Plot the simulated interventions ranking
sd.plots.plot_simulated_interventions(s, df_sol, intervention_effects, interval_type="percentile", confidence_bounds=.95, top_plot=top_plot);  # Plot the simulated interventions

## Feedback loop analysis with Loops That Matter

In [ ]:
#assert s.interaction_terms == False, "Interaction terms must be False for the feedback loop analysis."
int_var = None  # If set to None the highest-ranked intervention variable will be used
df_loops, loopscores_per_sample = sdm.run_loops_that_matter(int_var)  # Run the feedback loop analysis
cut_off_loop_importance = 0.05
sd.plots.plot_feedback_loops_ranking(s, df_loops, intervention_effects, int_var, cut_off_loop_importance);
sd.plots.plot_feedback_loops_over_time(s, df_loops, intervention_effects, loopscores_per_sample, int_var, cut_off_loop_importance);

## Sensitivity analysis

In [ ]:
## Estimate spearman correlation for the model parameters, either for one intervention variable (int_var) or for all
cut_off_SA_importance = 0.05
input_var = True  # If True, the top-ranked intervention variable will be used
outcome_var = False  # If True, the variable of interest will be used
SA_results, df_SA = sdm.run_SA(outcome_var, input_var, int_var, cut_off_SA_importance)